In [1]:
import numpy as np
import pandas as pd
from math import radians
from geopy import distance
import sys
from math import dist
from geopy.geocoders import Nominatim

In [2]:
#Create the dataframes based on the files
df1 = pd.read_csv('energy-and-water.csv')

df2 = pd.read_csv('Charging_Stations.csv')

In [22]:
#Select the relevant columns from the properties dataset
properties_df = df1[['Building Address ', "Building Address City", "Building Address Zip Code", 'Site EUI (Energy Use Intensity kBtu/ft²)', 'Total Site Energy Usage (kBtu)']]

In [23]:
# Rank the properties based on total energy usage and energy efficiency
properties_sorted = properties_df.sort_values(by='Total Site Energy Usage (kBtu)', ascending = False)

# Create a subset with the top 5% homes with biggest energy usage
top_5_percent_index = int(0.05 * len(properties_sorted))

properties_ranked = properties_sorted.head(top_5_percent_index)

properties_ranked

,Building Address,Building Address City,Building Address Zip Code,Site EUI (Energy Use Intensity kBtu/ft²),Total Site Energy Usage (kBtu)
1388,20 gillette park,boston,02127,615.5,899467175.8
4939,360 Huntington Avenue,boston,02115,118.5,814503655.4
102,5 75 francis st,boston,02115,351.0,571856204.5
5017,60 Fenwood Road & 70 Francis Street,boston,02115,370.6,457801845.7
2444,830-840 harrison ave,boston,02118,395.8,373714603.2
...,...,...,...,...,...
52,69-81 tremont st,boston,02108,70.2,21352711.4
260,157 berkeley street,boston,02116,28.0,21191519.2
87,175 berkeley st.,boston,02116,40.3,21051310.7
125,588-622 washington st,boston,02111,83.0,20873918.4


In [69]:
#Select the relevant columns from the stations dataset
stations_df = df2[["OBJECTID", "Street_Address", "City","State","ZIP","Latitude","Longitude"]]

In [25]:
#Find the coordinates for the zipcodes in the properties dataset to be able to measure the distances
geolocator = Nominatim(timeout=10, user_agent="PDS")

properties_ranked['geocode'] = properties_ranked['Building Address Zip Code'].apply(geolocator.geocode)

/var/folders/sw/x1bms4y570g2tql1p438nt0w0000gn/T/ipykernel_72949/4192275891.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  properties_ranked['geocode'] = properties_ranked['Building Address Zip Code'].apply(geolocator.geocode)


In [26]:
properties_ranked['Longitude'] = [g.longitude for g in properties_ranked.geocode]
properties_ranked['Latitude'] = [g.latitude for g in properties_ranked.geocode]

/var/folders/sw/x1bms4y570g2tql1p438nt0w0000gn/T/ipykernel_72949/2187826269.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  properties_ranked['Longitude'] = [g.longitude for g in properties_ranked.geocode]
/var/folders/sw/x1bms4y570g2tql1p438nt0w0000gn/T/ipykernel_72949/2187826269.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  properties_ranked['Latitude'] = [g.latitude for g in properties_ranked.geocode]


In [27]:
properties_ranked

,Building Address,Building Address City,Building Address Zip Code,Site EUI (Energy Use Intensity kBtu/ft²),Total Site Energy Usage (kBtu),geocode,Longitude,Latitude
1388,20 gillette park,boston,02127,615.5,899467175.8,"(Carhuaz, 02127, Ancash, Perú, (-9.28091044074...",-77.647368,-9.280910
4939,360 Huntington Avenue,boston,02115,118.5,814503655.4,"(Naujininkų seniūnija, Vilnius, 02115, Vilniau...",25.283215,54.665113
102,5 75 francis st,boston,02115,351.0,571856204.5,"(Naujininkų seniūnija, Vilnius, 02115, Vilniau...",25.283215,54.665113
5017,60 Fenwood Road & 70 Francis Street,boston,02115,370.6,457801845.7,"(Naujininkų seniūnija, Vilnius, 02115, Vilniau...",25.283215,54.665113
2444,830-840 harrison ave,boston,02118,395.8,373714603.2,"(Naujininkų seniūnija, Vilnius, 02118, Vilniau...",25.291376,54.663165
...,...,...,...,...,...,...,...,...
52,69-81 tremont st,boston,02108,70.2,21352711.4,"(Rasų seniūnija, Vilnius, 02108, Vilniaus mies...",25.289628,54.669898
260,157 berkeley street,boston,02116,28.0,21191519.2,"(Naujininkų seniūnija, Vilnius, 02116, Vilniau...",25.287308,54.663783
87,175 berkeley st.,boston,02116,40.3,21051310.7,"(Naujininkų seniūnija, Vilnius, 02116, Vilniau...",25.287308,54.663783
125,588-622 washington st,boston,02111,83.0,20873918.4,"(Naujininkų seniūnija, Vilnius, 02111, Vilniau...",25.285370,54.668014


In [50]:
#Add a column with unique id to the properties
properties_ranked['ID'] = properties_ranked.reset_index().index + 1

properties_ranked

/var/folders/sw/x1bms4y570g2tql1p438nt0w0000gn/T/ipykernel_72949/2036814276.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  properties_ranked['ID'] = properties_ranked.reset_index().index + 1


,Building Address,Building Address City,Building Address Zip Code,Site EUI (Energy Use Intensity kBtu/ft²),Total Site Energy Usage (kBtu),geocode,Longitude,Latitude,ID
1388,20 gillette park,boston,02127,615.5,899467175.8,"(Carhuaz, 02127, Ancash, Perú, (-9.28091044074...",-77.647368,-9.280910,1
4939,360 Huntington Avenue,boston,02115,118.5,814503655.4,"(Naujininkų seniūnija, Vilnius, 02115, Vilniau...",25.283215,54.665113,2
102,5 75 francis st,boston,02115,351.0,571856204.5,"(Naujininkų seniūnija, Vilnius, 02115, Vilniau...",25.283215,54.665113,3
5017,60 Fenwood Road & 70 Francis Street,boston,02115,370.6,457801845.7,"(Naujininkų seniūnija, Vilnius, 02115, Vilniau...",25.283215,54.665113,4
2444,830-840 harrison ave,boston,02118,395.8,373714603.2,"(Naujininkų seniūnija, Vilnius, 02118, Vilniau...",25.291376,54.663165,5
...,...,...,...,...,...,...,...,...,...
52,69-81 tremont st,boston,02108,70.2,21352711.4,"(Rasų seniūnija, Vilnius, 02108, Vilniaus mies...",25.289628,54.669898,248
260,157 berkeley street,boston,02116,28.0,21191519.2,"(Naujininkų seniūnija, Vilnius, 02116, Vilniau...",25.287308,54.663783,249
87,175 berkeley st.,boston,02116,40.3,21051310.7,"(Naujininkų seniūnija, Vilnius, 02116, Vilniau...",25.287308,54.663783,250
125,588-622 washington st,boston,02111,83.0,20873918.4,"(Naujininkų seniūnija, Vilnius, 02111, Vilniau...",25.285370,54.668014,251


In [51]:
#Convert coordinates into dictionaries
homes_coordinates = properties_ranked[["ID","Latitude","Longitude"]]
homes_dict = homes_coordinates.set_index('ID').T.to_dict('list')

In [52]:
stations_coordinates = stations_df[["OBJECTID","Latitude","Longitude"]]
stations_dict = stations_coordinates.set_index('OBJECTID').T.to_dict('list')

In [53]:
#Calculate the distance from the properties to the nearest station
latitude = []
longitude = []
Min_Distance = []
number = []

for [x,y] in homes_dict.values():
    min_distance=sys.maxsize
    for [p,q] in stations_dict.values():
        res = dist([x,y], [p,q])
        if res < min_distance:
            min_distance = res
            key1 = {i for i in stations_dict if stations_dict[i]==[p,q]}
            p_value = p
            q_value = q
            station_id = ''.join([str(a) for a in key1])
    latitude.append(p_value)
    longitude.append(q_value)
    Min_Distance.append(min_distance)
    number.append(station_id)
    key2 = {i for i in homes_dict if homes_dict[i]==[x,y]}     

In [54]:
#Create a data frame with the calculation of the distances between stations and properties
new_columns = {
    'Nearest_Station_Latitude': latitude,
    'Nearest_Station_Longitude': longitude,
    'Min_Distance': Min_Distance,
    'Nearest_Station_ID': number}

In [59]:
new_data = pd.DataFrame(new_columns)

In [60]:
new_data

,Nearest_Station_Latitude,Nearest_Station_Longitude,Min_Distance,Nearest_Station_ID
0,42.321069,-71.110515,52.014370,8
1,42.388734,-71.010123,97.072738,3
2,42.388734,-71.010123,97.072738,3
3,42.388734,-71.010123,97.072738,3
4,42.388734,-71.010123,97.080587,3
...,...,...,...,...
247,42.388734,-71.010123,97.079704,3
248,42.388734,-71.010123,97.076629,3
249,42.388734,-71.010123,97.076629,3
250,42.388734,-71.010123,97.075242,3


In [67]:
#Concatenate both dataframes to include property information, energy usage information and the calculated distances
homes = pd.concat([properties_ranked.reset_index(drop=True), new_data.reset_index(drop=True)], axis=1)

In [68]:
homes

,Building Address,Building Address City,Building Address Zip Code,Site EUI (Energy Use Intensity kBtu/ft²),Total Site Energy Usage (kBtu),geocode,Longitude,Latitude,ID,Nearest_Station_Latitude,Nearest_Station_Longitude,Min_Distance,Nearest_Station_ID
0,20 gillette park,boston,02127,615.5,899467175.8,"(Carhuaz, 02127, Ancash, Perú, (-9.28091044074...",-77.647368,-9.280910,1,42.321069,-71.110515,52.014370,8
1,360 Huntington Avenue,boston,02115,118.5,814503655.4,"(Naujininkų seniūnija, Vilnius, 02115, Vilniau...",25.283215,54.665113,2,42.388734,-71.010123,97.072738,3
2,5 75 francis st,boston,02115,351.0,571856204.5,"(Naujininkų seniūnija, Vilnius, 02115, Vilniau...",25.283215,54.665113,3,42.388734,-71.010123,97.072738,3
3,60 Fenwood Road & 70 Francis Street,boston,02115,370.6,457801845.7,"(Naujininkų seniūnija, Vilnius, 02115, Vilniau...",25.283215,54.665113,4,42.388734,-71.010123,97.072738,3
4,830-840 harrison ave,boston,02118,395.8,373714603.2,"(Naujininkų seniūnija, Vilnius, 02118, Vilniau...",25.291376,54.663165,5,42.388734,-71.010123,97.080587,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,69-81 tremont st,boston,02108,70.2,21352711.4,"(Rasų seniūnija, Vilnius, 02108, Vilniaus mies...",25.289628,54.669898,248,42.388734,-71.010123,97.079704,3
248,157 berkeley street,boston,02116,28.0,21191519.2,"(Naujininkų seniūnija, Vilnius, 02116, Vilniau...",25.287308,54.663783,249,42.388734,-71.010123,97.076629,3
249,175 berkeley st.,boston,02116,40.3,21051310.7,"(Naujininkų seniūnija, Vilnius, 02116, Vilniau...",25.287308,54.663783,250,42.388734,-71.010123,97.076629,3
250,588-622 washington st,boston,02111,83.0,20873918.4,"(Naujininkų seniūnija, Vilnius, 02111, Vilniau...",25.285370,54.668014,251,42.388734,-71.010123,97.075242,3
